In [ ]:
import EmbedQ
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import os
print(os.getcwd())

from model1 import EmotionLSTM  # or define EmotionLSTM inline if needed
import torch

path = r"C:\Users\sirid\Downloads\BobaLoversFinalCapstoneCogworks\best_model.pth"

print("File exists:", os.path.exists(path))
print("File size:", os.path.getsize(path), "bytes")

with open(path, 'rb') as f:
    start = f.read(10)
    print("File starts with:", start)

model = EmotionLSTM()
model.load_state_dict(torch.load(path))
model.eval()

surprise = 'Wait, what?! I didn\'t see that coming.'
disgust = "That\'s the grossest-looking pizza I've ever seen."
fear = "A chill ran down my spine when I saw the snake."
anger = "How could you do this?!"
sadness = "I feel really lonely"
happiness = "I\'m so excited for the trip next week, I\'m on top of the world right now!"

glove = EmbedQ.load_glove()

surprise = EmbedQ.embed_text(surprise, glove)
disgust = EmbedQ.embed_text(disgust, glove)
fear = EmbedQ.embed_text(fear, glove)
anger = EmbedQ.embed_text(anger, glove)
sadness = EmbedQ.embed_text(sadness, glove)
happiness = EmbedQ.embed_text(happiness, glove)

c:\Users\sirid\Downloads\BobaLoversFinalCapstoneCogworks\Text
File exists: True
File size: 3354466 bytes
File starts with: b'PK\x03\x04\x00\x00\x08\x08\x00\x00'
Loading GloVe embeddings (this may take a while)...


C:\Users\sirid\AppData\Local\Temp\ipykernel_4888\3882254141.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


GloVe 300d embeddings loaded successfully!


In [52]:
emotion_labels = ['happiness', 'sadness', 'anger', 'fear', 'surprise', 'disgust']

# Wrap each embedding in a list to create a batch with a single sample
surprise_batch = [surprise]
disgust_batch = [disgust]
fear_batch = [fear]
anger_batch = [anger]
sadness_batch = [sadness]
happiness_batch = [happiness]

def pad_batch(batch):
    padded_sequences = pad_sequence(batch, batch_first=True)
    lengths = torch.tensor([seq.size(0) for seq in batch])
    return padded_sequences, lengths

def get_predictions(dataloader):
    outputs_all = []
    with torch.no_grad():
        for inputs, lengths in dataloader:   # unpack exactly two variables: inputs and lengths
            # Debug print (optional):
            # print(f"inputs shape: {inputs.shape}, lengths: {lengths}")
            outputs = model(inputs, lengths)
            outputs_all.append(outputs)
    return torch.cat(outputs_all, dim=0)

surpriseDataloader = DataLoader(surprise_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)
disgustDataloader = DataLoader(disgust_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)
fearDataloader = DataLoader(fear_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)
angerDataloader = DataLoader(anger_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)
sadnessDataloader = DataLoader(sadness_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)
happinessDataloader = DataLoader(happiness_batch, batch_size=1, shuffle=False, collate_fn=pad_batch)

In [41]:
# Get predictions
surprisePredictions = get_predictions(surpriseDataloader)
disgustPredictions = get_predictions(disgustDataloader)
fearPredictions = get_predictions(fearDataloader)
angerPredictions = get_predictions(angerDataloader)
sadnessPredictions = get_predictions(sadnessDataloader)
happinessPredictions = get_predictions(happinessDataloader)

# Print results
print("Surprise Predictions:", surprisePredictions)
print("Disgust Predictions:", disgustPredictions)
print("Fear Predictions:", fearPredictions)
print("Anger Predictions:", angerPredictions)
print("Sadness Predictions:", sadnessPredictions)
print("Happiness Predictions:", happinessPredictions)

Surprise Predictions: tensor([[ 0.7235,  0.8699,  0.0726,  0.0186,  0.0047, -0.0144]])
Disgust Predictions: tensor([[0.5053, 0.6821, 0.0967, 0.0387, 0.0114, 0.0176]])
Fear Predictions: tensor([[0.3996, 0.7093, 0.0686, 0.0544, 0.0250, 0.0254]])
Anger Predictions: tensor([[ 5.5187e-01,  7.4286e-01,  8.2257e-02,  3.5643e-02,  2.1396e-04,
         -1.7214e-02]])
Sadness Predictions: tensor([[ 0.6224,  0.7794,  0.0624,  0.0186,  0.0042, -0.0020]])
Happiness Predictions: tensor([[0.2332, 0.4918, 0.1107, 0.1216, 0.0115, 0.0297]])


In [ ]:
def inputToPred(input_text):

    glove = EmbedQ.load_glove()
    dummy_label = torch.zeros(6)  
    embedded = [(EmbedQ.embed_text(input_text, glove), dummy_label)]
    print(embedded[0].shape)
    model = model1.EmotionLSTM()
    model.load_state_dict(torch.load("best_model.pth"))

    # DataLoader setup  
    textDataLoader = DataLoader(embedded, batch_size=1, shuffle=False, collate_fn=pad_batch)

    model.eval()

    with torch.no_grad():
        for inputs, lengths, _ in textDataLoader:
            outputs = model(inputs, lengths)  
            pred_vector= outputs.squeeze(0)      

    result = {
        emotion_labels[i]: round(pred.item(), 3)
        for i, pred in enumerate(pred_vector)
        if pred.item() > 0
    }
    return result

print(inputToPred("I am happy"))

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn

# Emotion labels matching the model output order
emotion_labels = ['happiness', 'sadness', 'anger', 'fear', 'surprise', 'disgust']

# Example class weights — adjust these based on your needs:
# Lower weight reduces the logit impact (penalizes),
# Higher weight increases the logit impact (boosts)
class_weights = torch.tensor([
    0.7, 
    0.6,  
    1.1, 
    1.1,  
    1.3,  
    0.7   
], dtype=torch.float32)

# Assuming EmbedQ and glove are already loaded and model is initialized
# Criterion if needed (not used here since it's inference)
criterion = nn.BCEWithLogitsLoss()


def preprocess(text):
    # Embed text into tensor sequence [seq_len, 300]
    embedded = EmbedQ.embed_text(text, glove)
    return embedded


def prepare_batch(tensors):
    lengths = torch.tensor([t.size(0) for t in tensors])
    padded = pad_sequence(tensors, batch_first=True)  # [batch_size, max_seq_len, 300]
    return padded, lengths


def predict(texts_dict, model, class_weights):
    model.eval()
    results = {}

    # Move weights to same device as model parameters if needed
    device = next(model.parameters()).device
    weights = class_weights.to(device)

    for label, text in texts_dict.items():
        embedded = preprocess(text).to(device)
        padded, lengths = prepare_batch([embedded])
        padded = padded.to(device)
        lengths = lengths.to(device)

        with torch.no_grad():
            outputs = model(padded, lengths)  # logits shape: [batch_size, num_emotions]
            logits = outputs[0]  # for single sample in batch

            # Apply element-wise class weights to logits
            weighted_logits = logits * weights

            # Convert weighted logits to probabilities
            scores = torch.sigmoid(weighted_logits)

            predicted_emotions = [(emotion_labels[i], scores[i].item()) for i in range(len(emotion_labels))]

            results[label] = {
                'text': text,
                'predicted_emotions': sorted(predicted_emotions, key=lambda x: x[1], reverse=True),
                'weighted_logits': weighted_logits.tolist(),
                'confidence_scores': scores.tolist()
            }

    return results

# Use the predict function
results = predict(texts, model, class_weights)

# Print formatted results
for true_label, data in results.items():
    print(f"\nInput Label: {true_label}")
    print(f"Text: {data['text']}")
    print("Predicted Emotions:")
    if data['predicted_emotions']:
        for emotion, score in data['predicted_emotions']:
            print(f"  - {emotion}: {score:.4f}")
    else:
        print("  - None above threshold (0.5)")
    print(f"Weighted Logits: {data['weighted_logits']}")



Input Label: surprise
Text: Wait, what?! I didn't see that coming.
Predicted Emotions:
  - sadness: 0.6276
  - happiness: 0.6240
  - anger: 0.5199
  - fear: 0.5051
  - surprise: 0.5015
  - disgust: 0.4975
Weighted Logits: [0.5064367055892944, 0.5219403505325317, 0.07982942461967468, 0.020457318052649498, 0.006166641600430012, -0.010072704404592514]

Input Label: disgust
Text: That's the grossest-looking pizza I've ever seen.
Predicted Emotions:
  - sadness: 0.6009
  - happiness: 0.5875
  - anger: 0.5266
  - fear: 0.5106
  - surprise: 0.5037
  - disgust: 0.5031
Weighted Logits: [0.3536751866340637, 0.4092780649662018, 0.10633145272731781, 0.04256168007850647, 0.014793332666158676, 0.012317330576479435]

Input Label: fear
Text: A chill ran down my spine when I saw the snake.
Predicted Emotions:
  - sadness: 0.6048
  - happiness: 0.5695
  - anger: 0.5188
  - fear: 0.5150
  - surprise: 0.5081
  - disgust: 0.5044
Weighted Logits: [0.2797470688819885, 0.42556437849998474, 0.0754127800464630